<div style="background-color:rgba(255, 0, 0, 0.5); padding: 10px; border-radius: 5px; width: 95%">

### 🔒 **Limited Autonomy of Large Language Models (LLMs)**

Large Language Models (LLMs) operate based on static training data and do **not** have real-time awareness or built-in access to current events, live systems, or external APIs.  
Unless explicitly integrated with such tools, they:

- Cannot browse the web  
- Cannot run code  
- Cannot interact with real-time data sources

As a result, when asked questions that require up-to-date knowledge or external action, LLMs may:

- Provide outdated, inaccurate, or fabricated responses, **or**
- Acknowledge their limitations and defer the question
</div>

<div style="background-color:rgba(0, 0, 0, 0.5); padding: 10px; border-radius: 5px; width: 95%">

### 🚀 **Purpose of This Notebook**

This notebook explores how to **extend the capabilities** of LLMs by connecting them to external tools and APIs.  
By integrating real-time services, we can:

- Enable live data access  
- Perform dynamic computations  
- Interact with the environment beyond the training corpus

The goal is to demonstrate practical techniques for augmenting LLMs with **tool use**, turning them from static responders into dynamic, action-capable assistants.

</div>


# Start Ollama Server

1. Open a terminal

    ☰ > *Terminal* > *New Terminal*

2. Start the Ollama server 

    ```bash
    ollama serve
    ```

3. Do not close this terminal

4. Open another terminal
5. Pull the model *"qwen3:4b"* (if not already done)

    ```bash
    ollama pull qwen3:4b
    ```
5. Close this terminal



# Define Environment Variables

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen3:4b"
LLM_SEED = 42 
LLM_TEMPERATURE = 0.0
TEST_PROMPT0 = "What time is it?"
TEST_PROMPT1 = "What is the price of gold right now?" 
TEST_PROMPT3 = "How many 1 in 111111111111111?"

# Initialize Ollama Chatbot

In [ ]:
from langchain_ollama import ChatOllama

# Set up the Ollama chat model with specified LLM model and parameters
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    seed=LLM_SEED,
    stream=True
)

# Define Tools

1. **Python Tool:** This tool allows the chatbot to execute Python code, enabling it to perform calculations, data processing, and other tasks that require programming logic.

2. **Commodity Price Tool:** This tool fetches real-time commodity prices from the `API Ninjas` API, allowing the chatbot to provide up-to-date information on various commodities.
    - Go to [``API Ninjas``](https://api-ninjas.com/)
    - Click the "Sign Up" button
    - Create an account
    - Log in
    - Click the "My Account" button
    - Click the "Show API Key" button

    In the next cell, you will need to:
    - Replace the content of the variable `NINJA_API_KEY` with your API key.
    - Run the cell to test the API tool

In [ ]:
from langchain_core.tools import tool, Tool
from typing import Annotated, List
import io
import contextlib
import requests
import urllib3

NINJA_API_KEY = "MdaCcVUHseixkwpNGhumWg==9mb676ZThl4du9aP"

@tool
def execute_python(py_code: Annotated[str, "Python code to execute"]) -> str:
    """Executes a Python code and returns its standard output (you have to use the print() function)."""
    output = io.StringIO()
    try:
        with contextlib.redirect_stdout(output):
            exec(py_code, {})
        return output.getvalue().strip() or "Code executed with no output."
    except Exception as e:
        return f"Error: {str(e)}"

# Test the tool
print(execute_python.invoke("print('Hello, World!')"))

@tool
def get_commodity_price(
    commodity: Annotated[str, "The name of the commodity to get the price for"]
) -> str:
    """Get the current price of a commodity using the Ninja API."""
    api_url = 'https://api.api-ninjas.com/v1/commodityprice?name={}'.format(commodity)
    response = requests.get(api_url, headers={'X-Api-Key': NINJA_API_KEY})
    if response.status_code == requests.codes.ok:
        return response.text
    else:
        return f"Error {response.status_code}: {response.text}"

#Test the tool
print(get_commodity_price.invoke("gold"))
    
tools = [execute_python, get_commodity_price]




# Prepare Reason-and-Act (ReAct) Instructions

ReAct (Reason-and-Act) prompting combines:

- **Reasoning:** The model explains its thinking process.
- **Acting:** The model chooses and performs actions (like using a calculator, web search, or database query).

The prompt encourages the model to alternate between these two steps, creating a loop:

→ Think → Act → Observe → Think → Act → … until the task is complete.

**Reference:**

(Yao et al., 2023) Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2023, January). ReAct: Synergizing reasoning and acting in language models. In *International Conference on Learning Representations (ICLR)*.]{https://doi.org/10.48550/arXiv.2210.03629}

In [ ]:
from langchain_core.runnables import RunnableLambda
from inspect import signature

tools_names = ", ".join([tool.name for tool in tools])
tools_descriptions = "\n".join([f"{tool.name}{signature(tool.func)} - {tool.description}" for tool in tools]) 


# Taken from https://smith.langchain.com/hub/hwchase17/react-json
instructions = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherwise, you have access to the following tools:

{tools_descriptions}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tools_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer:` when responding.
Always make your final answer easy to read and understand for humans.
"""

print(instructions)

# Test the Chatbot

In [ ]:
from src.Chatbot import Chatbot
from langchain_core.messages import SystemMessage
# Create a chat history with a system message and a human message
chatbot = Chatbot(llm=llm, history=[SystemMessage(content=instructions)])
chatbot.invoke(TEST_PROMPT0)
# chatbot.interact()

<div style= "padding: 0.5em;background-color: rgba(255,0,0, 0.5);width: 95%">

### **Problem:** the LLM generated the observation and should have waited for the tool response

Here, we did not interrupt the LLM when it generated the keyword *"Observation:"*. Meaning that we did not actually call the tool. Hence, the LLM hallucinated and made up a response.
</div>

# Interrupting the chatbot when generating '*Observations:*'

In [ ]:

binded_llm = llm.bind(stop=["Observation:"])
history = [SystemMessage(content=instructions)]
chatbot = Chatbot(llm=binded_llm, history=history)
#chatbot.invoke(TEST_PROMPT0)
#chatbot.interact()

# Create an Agent Class

In [ ]:
from src.Agent import Agent 

# Testing the Agent

In [ ]:
# Create a chat history with a system message and a human message
agent = Agent(llm=llm, tools=tools, history=[SystemMessage(content=instructions)])
agent.invoke(TEST_PROMPT0)
agent.invoke(TEST_PROMPT1)

### Bonus

To go further, you can explore Model Context Protocol (MCP) which allows you to make your tools and APIs publicly available to the LLMs.
